This Notebook is my exercise1 version. I have adapted the week 1 exercise solution to an article's abstract summariser using from Europe PMC's article API. [Europe PMC (EPMC)](https://europepmc.org/) is a free, open-access database that provides access to a wealth of life sciences and biomedical literature. It is an integral part of the PubMed Central International (PMCI) network, aggregating content from multiple sources and offering access to millions of scientific articles, research papers, and preprints, all in one place. 

My solution uses a provided article's PMCID(obtainable by selecting an article you wish to summarise from EPMC's website). PMCID are unique only to open-access articles and you can only use the function below for such articles. To get an article's PMICD: 
To use:
1. Go to [EPMC's Website](https://europepmc.org/)
1. Use the search tab and search for articles by keywords, entities journal or Author's name. E.g Genes, Diseases, nutrition etc
2. Search for open-access articles by including the keyword `HAS_FT:Y` or `IN_EPMC:Y`. Example: `"Genes: HAS_FT:Y"`
3. Then your article of interest and copy the PMCID. 
4. feed the PMCID into the `display_reponse` func to generate the summary from the article's abstract.   

In [ ]:
import re
import pprint
from pprint import pformat
import requests
import functools
from typing import List, Tuple, Dict, Any

from tqdm import tqdm
from loguru import logger
from bs4 import BeautifulSoup as bs

from IPython.display import display, HTML, Markdown

import ollama

In [ ]:
def catch_request_error(func):
    """
    Wrapper func to catch request errors and return None if an error occurs.
    Used as a decorator.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except requests.RequestException as e:
            print(f"Request error in {func.__name__}: {e}")
            return None
    return wrapper

In [ ]:
@catch_request_error
@logger.catch
def get_xml_from_url(url: str) -> bs:
    """
    Fetches the XML content from Europe PMC website.

    Args:
        url (str): Europe PMC's production url to fetch the XML from.

    Returns:
        soup (bs4.BeautifulSoup): Parsed XML content.
    """
    response = requests.get(url)
    response.raise_for_status() #check for request errors
    return bs(response.content, "xml")  

In [ ]:
def clean_text(text:str) -> str:
    """
    This function cleans a text by filtering reference patterns in text, 
    extra whitespaces, escaped latex-style formatting appearing in text body instead of predefined latex tags

    Args: 
    text(str): The text to be cleaned
    
    Returns: 
    tex(str): The cleaned text 
    
    """
   
    # Remove LaTeX-style math and formatting tags #already filtered from soup content but some still appear
    text = re.sub(r"\{.*?\}", "", text)  # Matches and removes anything inside curly braces {}
    text = re.sub(r"\\[a-zA-Z]+", "", text)  # Matches and removes characters that appears with numbers
    
    # Remove reference tags like [34] or [1,2,3]
    text = re.sub(r"\[\s*(\d+\s*(,\s*\d+\s*)*)\]", "", text)
    
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


def fetch_article_abstract(soup: bs) -> Tuple[str, str]:
    """
    Extracts the abstract text from the XML soup.

    Args:
        soup (bs4.BeautifulSoup): Parsed XML content.
    Returns:
        Tuple(article_title (str), abstract_text (str)): A tuple of the article's title and its extracted abstract text.
    """
    if soup is None:
        return "No XML found", ""
    article_title = soup.find("article-title").get_text(strip=True) if soup.find("article-title") else "No Title Found for this article"

    abstract_tag = soup.find("abstract")
    if abstract_tag:
        abstract_text = ' '.join([clean_text(p.get_text(strip=True)) for p in abstract_tag.find_all("p") if p.get_text(strip=True)])
    else:
        abstract_text = ""
    return article_title, abstract_text

In [ ]:
sys_prompt = """You are an expert in biomedical text mining and information extraction. 
You excel at breaking down complex articles into digestible contents for your audience. 
Your audience can comprise of students, early researchers and professionals in the field.
Summarize the key findings in the following article [ARTICLE] .
Your summary should provide crucial points covered in the paper that helps your diverse audience quickly understand the most vital information. 
Crucial points to consider:
- Main objectives of the study
- Key findings and results
- Methodologies used
- Implications of the findings(if any)
- Any limitations or future directions mentioned

Format: Provide your summary in bullet points highlighting key areas followed with a  concise paragraph that encapsulates the results of the paper.

The tone should be professional and clear.

"""

In [ ]:
MODEL = "llama3.2"

In [ ]:
def build_message(article_title: str, abstract_text: str, sys_prompt:str=sys_prompt) -> List[Dict[str, str]]:
    """
    Constructs the message payload for the LLM.

    Args:
        article_title (str): The title of the article.
        abstract_text (str): The abstract text of the article.

    Returns:
        List[Dict[str, str]]: A list of message dictionaries for the LLM.
    """
    user_prompt = f"""You are looking at an article with title:  {article_title}. 
    The article's abstract is as follows: \n{abstract_text}.
    Summarise the article. Start your summary by providing a short sentence on what the article is about 
    and then a bulleted list of the key points covered in the article.
"""
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    return messages

In [ ]:
def generate_response(messages, model=MODEL):
    response = ollama.chat(model=model, messages=messages)
    return response["message"]["content"]

In [ ]:
#combine everything to main function
def display_reponse(article_id:str):
    if article_id and not re.match(r"^PMC\d{5,8}$", article_id):
        raise ValueError("Please check the length/Format of the provided Article ID. It should start with 'PMC' followed by 5 to 8 digits, e.g., 'PMC1234567'.")
    url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{article_id}/fullTextXML"
    soup = get_xml_from_url(url)
    article_title, abstract_text = fetch_article_abstract(soup)
    messages = build_message(article_title, abstract_text)
    response = generate_response(messages)

    display(Markdown(f"### Article Title: {article_title}"))
    display(Markdown(f"### LLM Response: \n{response}"))

In [ ]:
#add your article's PMCID here to test the function
display_reponse("PMC7394925")

In [ ]:
display_reponse("PMC12375411")